In [1]:
import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets

prepare mnist dataset

In [2]:
(train_x, train_y), (test_x, test_y) = datasets.mnist.load_data()

In [3]:
inputs = layers.Input((28, 28, 1))
#feature extraction
net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(0.25)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(0.25)(net)


# fully connected
net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(0.5)(net)
net = layers.Dense(10)(net)  # num_classes
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
model.summary()

Model: "Basic_CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 32)        0 

Optimizaion

모델을 학습하기 전 설정

- loss function
- optimization
- Metrics

Loss Function

loss function 방법확인

Categorical vs Binary

In [5]:
tf.keras.losses.binary_crossentropy

<function tensorflow.python.keras.losses.binary_crossentropy>

In [6]:
loss = 'binary_crossentropy' # 있다 없다
loss = 'categorical_crossentropy' # 2개 이상

sparse_categorical_crossentropy vs categorical_crossentropy

In [7]:
tf.keras.losses.sparse_categorical_crossentropy # onehot encoding 안했을 경우

# [1,2,3]

<function tensorflow.python.keras.losses.sparse_categorical_crossentropy>

In [8]:
tf.keras.losses.categorical_crossentropy # onehot encoding 한 경우

<function tensorflow.python.keras.losses.categorical_crossentropy>

Metrics
모델을 평가하는 방법

accuracy를 이름으로 넣는 바법

In [9]:
metrics = ['accuracy']

In [10]:
tf.keras.metrics.Accuracy()
tf.keras.metrics.Precision()
tf.keras.metrics.Recall()

Compile

optimizer 적용

- 'sgd'
- 'rmsprop'
- 'adam'

In [11]:
tf.keras.optimizers.Adam()


- tf.keras.optimizers.SGD()
- tf.keras.optimizers.RMSprop()
- tf.keras.optimizers.Adam()

metrics에서 tf.keras.metrics.Accuracy()과 sparse_categorical_crossentropy와 충돌이 일어난다 그래서
tf.keras.metrics.sparse_categorical_accuracy으로 사용해야한다

In [12]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=[tf.keras.metrics.sparse_categorical_accuracy])

prepare dataset

학습에 사용할 데이터셋 준비

shape 확인

In [14]:
train_x.shape, train_y.shape

((60000, 28, 28), (60000,))

In [15]:
test_x.shape, test_y.shape

((10000, 28, 28), (10000,))

차원수 늘리기

In [16]:
import numpy as np

In [17]:
np.expand_dims(train_x, -1).shape

(60000, 28, 28, 1)

In [18]:
train_x = train_x[...,tf.newaxis]
test_x = test_x[..., tf.newaxis]

차원수 잘 늘었는지 확인

In [19]:
train_x.shape, test_x.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

rescaling - 학습이 튀지 않고 학습이 잘되고 기울기가 잘 잡힌다


In [20]:
np.min(train_x), np.max(train_x)

(0, 255)

In [21]:
train_x = train_x/255.
test_x = test_x/255.

In [22]:
np.min(train_x), np.max(train_x)

(0.0, 1.0)

Training

본격적으로 학습 들어가기

학습용 hyperparameter 설정
- num_epochs
- batch_size


In [23]:
num_epochs = 1
batch_size = 32

- model.fit()

In [26]:
model.fit(train_x, train_y, batch_size=batch_size, shuffle=True, epochs=1)

1875/1875 [==============================] - 264s 141ms/step - loss: 0.1494 - sparse_categorical_accuracy: 0.9539
